## Wire Delay

We can execute the Perl script `WireDelay.pl` by calling the Perl interpreter from the command line, providing it the name of the script as well as any parameters that it needs to work. That is, we want to run

`$ perl ./perl/WireDelay.pl <threshIn> <outputs[0]> <geoDir> <daqId> <fw>`

where the items in angled brackets `<>` are specific parameters we input for a specific set of data.  For example, we might want

* threshIn = 'files/6119.2016.0104.1.thresh'
* outputs[0] = '6119.2016.0104.1.wd'
* geoDir = './geo'
* daqId = '6119'
* fw = '1.12'

in which case the command is

`$ perl ./perl/WireDelay.pl files/6119.2016.0104.1.thresh 6119.2016.0104.1.wd ./geo 6119 1.12`

We can use the shell command `head` to get a quick look at what the input file `6119.2016.0104.1.thresh` looks like:

In [1]:
!head -6 files/6119.2016.0104.1.thresh

#$md5
#md5_hex(0)
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6119.1	2457392	0.3721863017828993	0.3721863017831598	22.50	3215689647404250	3215689647406500
6119.3	2457392	0.3721863017829138	0.3721863017831598	21.25	3215689647404375	3215689647406500
6119.2	2457392	0.3721885846820747	0.3721885846822772	17.50	3215709371653125	3215709371654875


Running more complicated terminal commands from inside a Notebook is hard (this is supposed to be an environment for Python, after all), but if you were to run the above `perl` command in a terminal after making sure the inputs `files/6119.2016.0104.1.thresh` and `geo/6119/6119.geo` are in place, you'd find that `WireDelay.pl` creates a file `6119.2016.0104.1.wd` that looks like

In [3]:
!head -6 6119.2016.0104.1.wd-shell

#USING WIREDELAYS: ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec)
6119.1	2457392	0.3721978758576562	0.3721978758579167	22.50
6119.3	2457392	0.3721978758576707	0.3721978758579167	21.25
6119.2	2457392	0.3722001587568317	0.3722001587570342	17.50
6119.4	2457392	0.3722001587568317	0.3722001587570486	18.75
6119.4	2457392	0.3722017606909173	0.3722017606911343	18.75


(I've renamed it `6119.2016.0104.1.wd-shell` to distinguish it from the similar files that will be created by running the command using Parsl from inside this Notebook)

So, what has `WireDelay.pl` script done to our data?  That is, how does `6119.2016.0104.1.wd` differ from `6119.2016.0104.1.thresh`?

First, we can see that the data appears to be in the same order.  The counters are recorded in the same order `6119.1, 6119.3, 6119.2, ...` and with the same "time over threshold" values `22.50, 21.25, 17.50, ...` in both files.  If you're not convinced, increase the number of lines returned by the `head` command until the pattern is clear.

Next, we note that `WireDelay` keeps the Julian day value from the Threshold file, but it drops the raw-integer rising and falling edge values.  Evidently we won't be needing these values for further analysis.

Last, we notice the most interesting thing: the rising and falling edge time values have changed!

Investigating further, we can see that for each line of data, the file `6119.2016.0104.1.wd` has values that are exactly `0.0000115740747569` greater than the corresponding value in the input file `6119.2016.0104.1.thresh`.  This time represents the *wire delay*: After a counter registers a hit, it takes a small amount of time for that signal to travel through the connector cable to the DAQ board, where it's recorded as data. The `WireDelay.pl` script adjusts the data for this, helping us make time comparisons between detectors as precisely as possible.

But how does `WireDelay.pl` know how much to adjust the rising and falling edge times by?  The answer is in the other input file, `6119.geo`.

### Now try it with Parsl!

To use the Parslized scripts, we'll first prepare Parsl:

In [ ]:
import parsl
from parsl import *

local_config = {
    "sites" : [
        { "site" : "Threads",
          "auth" : { "channel" : None },
          "execution" : {
              "executor" : "threads",
              "provider" : None,
              "max_workers" : 4
          }
        }],
    "globals" : {"lazyErrors" : True}
}

dfk = DataFlowKernel(config=local_config)

We write a Python function to return the shell command we saw above, and then we turn that function into a Parsl App with the `@App` decorator:

In [ ]:
@App('bash', dfk)
def WireDelay(threshIn='', outputs=[], geoDir='', daqId='', fw='', stdout='stdout.txt', stderr='stderr.txt'):
        return 'perl ./perl/WireDelay.pl %s %s %s %s %s' %(threshIn,outputs[0],geoDir,daqId,fw)